## Aufgabe 3 - Filter implementieren

In dieser Aufgabe werden Sie einen Filter auf eine Audiodatei anwenden.

Zunächst laden wir die Datei:

In [ ]:
# Verändern Sie diesen Block nicht
import numpy as np
import matplotlib.pyplot as plt
import wave
from IPython.display import Audio
with wave.open("Media1.WAV") as wav_file:
    sample_rate = wav_file.getframerate()
    n_samples = wav_file.getnframes()
    audio = np.frombuffer(wav_file.readframes(n_samples), dtype=np.int16)
audio = audio.reshape(-1, 2)[:, 0]  # Datei ist ursprünglich Stereo, wir brauchen nur einen Kanal
audio = audio.astype(np.double) / 32767
Audio(audio, rate=sample_rate)

Als nächstes Konstruieren wir den Filter. Für dieses Beispiel wollen wir einen naiven Low-Pass Filter mit einer Grenzfrequenz von 1000 Hz anwenden. In der Frequenzdomäne ist dieser leicht darzustellen:

In [ ]:
# Verändern Sie diesen Block nicht
freq_filter = np.zeros(1024)
x = np.linspace(-sample_rate / 2, sample_rate / 2, 1024)
freq_filter[abs(x) <= 1000] = 1
plt.plot(x, freq_filter)
plt.show()

Es ist einfach eine Funktion $F(\omega) = \begin{cases} 1 &\text{falls } |\omega| \leq 1000Hz \\ 0 &\text{sonst}\end{cases}$

Für die diskrete Faltung benötigen wir den Filter aber in der Zeitdomäne. Dafür wenden wir die inverse Fouriertransformation an und erhalten:

$f(t) = \frac{1}{\pi t}\sin(1000Hz\cdot t)$

Diese Funktion ist die Impulsantwort des Filters. Aber Sie hat ein Problem: Es gibt keinen Wert $t$, ab dem die Funktion immer 0 ist, also $\not \exists t: \forall t' > t: f(t) = 0$. Die Impulsantwort ist undendlich lang in der Zeit, wir sprechen von einem Infinite Impulse Response (IIR) Filter. Folglich wäre auch eine Faltung zwischen dem Filter und einem Signal unendlich lang.

In dieser Übung lösen wir das Problem, indem wir den Filter einfach abschneiden. Das verfälscht den Filter in der Frequenzdomäne, aber das nehmen wir hier einfach hin.

Wir begrenzen den Filter auf 512 samples in der Zeitdomäne. Die 1000Hz aus der Definition müssen wir in Abhängigkeit der Nyquist-Frequenz, also $\frac{\text{samplerate}}{2}$ ausdrücken.

In [ ]:
# Verändern Sie diesen Block nicht
t = np.arange(1, 513)
f = np.sin(t * 1000 / (sample_rate / 2)) / (np.pi * t)

Jetzt müssen Sie die Faltung implementieren.

a) Wie lang ist die diskrete Faltung zwischen zwei Signalen a und b? Die Länge der Signale erhalten Sie jeweils mit `len(a)` und `len(b)`. (5 Punkte)

b) Implementieren Sie die diskrete Faltung entsprechend der Definition:

$(a \ast b)[i] = \sum_{j=-\infty}^\infty a[j]b[i-j]$

Verwenden Sie **nicht** `np.convolve`. Sie dürfen andere `numpy` Funktionen, wie z.B. `np.sum` benutzen. (20 Punkte)

In [ ]:
def convolve(a, b):
    result = np.zeros(<Ihr Ergebnis aus a)>)
    
    # Implementieren Sie hier die Faltung
    
    return result

Sie können Ihre Implementierung zunächst an einem Ausschnitt der Daten testen und mit `np.convolve` vergleichen.

In [ ]:
# Verändern Sie diesen Block nicht
your_result = convolve(audio[:1000], f)

Falls der Folgende Block **keine** Ausgabe erzeugt, ist Ihr Ergebnis identisch mit `np.convolve`.

In [ ]:
# Verändern Sie diesen Block nicht
numpy_result = np.convolve(audio[:1000], f)
np.testing.assert_allclose(numpy_result, your_result)

Jetzt wollen wir die gesamte Datei filtern. Vergleichen Sie das Ergebnis mit der Originaldatei: Der Klang sollte jetzt dumpf sein, wie durch eine Wand.

In dieser Abgabe bewerten wir nur die Korrektheit Ihrer Lösung, nicht die Performance, deswegen bekommen Sie keinen Punktabzug, wenn Ihre Lösung nicht auf die ganze Audiodatei angewendet werden kann.

In [ ]:
# Verändern Sie diesen Block nicht
your_result = convolve(audio, f)
Audio(your_result, rate=sample_rate)

Eine Wichtige Erkenntnis ist der Zusammenhang zwischen der Faltung und der Fouriertransformation: Eine Faltung im Zeitbereich ist Äquivalent zu einer Multiplikation im Frequenzbereich. Durch die Verwendung des Fast Fourier Transform (FFT) Algorithmus ist es möglich, die Faltung deutlich zu beschleunigen.

Den FFT Algorithmus müssen Sie nicht selbst implementieren. Verwenden Sie `np.fft.fft` für die Fouriertransformation und `np.fft.ifft` für die inverse Fouriertransformation.

c) Implementieren Sie die Faltung, indem Sie Signal und Filter in den Frequenzbereich übertragen, multiplizieren und zurück übertragen. (15 Punkte)

Hinweis: Für $n$ Eingabepunkte liefert FFT auch wieder $n$ Ausgabepunkte. Sie müssen also ggf. die Arrays mit Nullen auffüllen, um die Längen anzugleichen. Benutzen Sie dazu entweder `np.concatenate` und `np.zeros` oder das Argument `n` von `np.fft.[i]fft`.

In [ ]:
def fft_convolve(a, b):
    # Implementieren Sie hier die Faltung

In [ ]:
# Verändern Sie diesen Block nicht
fft_result = fft_convolve(audio[:1000], f)
np.testing.assert_allclose(numpy_result, fft_result)

In [ ]:
# Verändern Sie diesen Block nicht
fft_result = fft_convolve(audio, f)
Audio(fft_result, rate=sample_rate)